###  Import DataSet and Preprocess

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path1='/content/drive/My Drive/Train.csv'
path2='/content/drive/My Drive/Test.csv'

In [ ]:
df_train=pd.read_csv(path1)
df_train.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,NaN,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.00,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,NaN,NaN,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.19,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


### Fill missing values with -999

In [ ]:
df_train.fillna(-999, inplace=True)

## Handle Categorical Variables with one hot encoding


In [ ]:
dum=pd.get_dummies(df_train['form_field47'])
df_train=pd.concat([df_train, dum], axis=1)

In [ ]:
df_train.drop('form_field47', axis=1, inplace=True)

In [ ]:
#yes:1, no: 0
df_train['default_status'].replace('no', 0, inplace=True)
df_train['default_status'].replace('yes', 1, inplace=True)

## Prepare Dataset for Modelling

In [ ]:
X=df_train.drop(['Applicant_ID','default_status'], axis=1)
Y=df_train['default_status']

## Import and Preprocess Test Dataset for Modeling

In [ ]:
df_test = pd.read_csv(path2)
df_test.head()


,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,0.0,1649749.0,5446.0,0.0,48187375,114686147.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1086.0,6204.0,2676.0,60.0,14691.0,9794.0,NaN,238.0,1.8334,0.0,NaN,22.0,2.0,14.0,0.0,NaN,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,18308285.0,622789.0,1025793.0,1396510.0,34694910,87479487.0,1.0,1.0,1.0,2.0,2.0,142.560,NaN,NaN,570.585,3082.5,10221.0,6814.0,60.0,11955.0,3528.0,NaN,1216.0,19.5000,3.0,NaN,30.0,18.0,14.0,2.0,NaN,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,0.0,118256.0,NaN,NaN,21973443,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,852.0,4836.0,NaN,NaN,NaN,7.8334,0.0,NaN,2.0,NaN,0.0,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,493641.0,1391064.0,1176725.0,1220293.0,16576457,NaN,2.0,2.0,2.0,3.0,3.0,109.005,13050.00,NaN,109.005,1414.5,4197.0,2798.0,60.0,14964.0,NaN,NaN,248.0,9.0000,1.0,1.0,36.0,0.0,26.0,0.0,NaN,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,5752921.0,271133.0,613064.0,4779643.0,48187375,72049802.0,0.0,0.0,0.0,3.0,3.0,139.065,469.29,469.29,15.180,2724.0,9855.0,6570.0,182.0,11679.0,5900.0,NaN,2388.0,16.6666,1.0,0.0,24.0,28.0,20.0,0.0,NaN,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


In [ ]:
df_test.fillna(-999,inplace=True)
dum=pd.get_dummies(df_test['form_field47'])
df_test=pd.concat([df_test, dum], axis=1)
df_test.drop('form_field47', axis=1, inplace=True)


In [ ]:
df_model=df_test.drop('Applicant_ID', axis=1)

In [ ]:
df_train.info()

FEATURE ENGINEERING (Log Transformation of some skewed columns)

In [ ]:
skewed_columns=['form_field14','form_field13','form_field7','form_field10','form_field50','form_field11', 'form_field48','form_field24','form_field9', 
              'form_field5','form_field38','form_field20','form_field34','form_field4','form_field3','form_field49', 'form_field19'] 


In [ ]:
## Transforming skewed columns in train dataset
import numpy as np
for i in skewed_columns:
    unskewed=np.log(X[i] + 1)
    X[i+'unskewed']=unskewed
    continue

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
## Transforming skewed columns in test dataset
for i in skewed_columns:
    unskewed=np.log(df_model[i] + 1)
    df_model[i+'unskewed']=unskewed
    continue

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
# Drop skewed columns and fill Nan with -999
for i in skewed_columns:
  X.drop(i, axis=1, inplace=True)
  df_model.drop(i, axis=1, inplace=True)
  continue

df_model.fillna(-999, inplace=True)
X.fillna(-999, inplace=True)

# Modelling


In [ ]:
!pip install catboost

In [ ]:
!pip install rgf_python

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold 
from sklearn.ensemble import RandomForestClassifier
from rgf.sklearn import RGFClassifier
rf=RandomForestClassifier(n_estimators=500)
rgf=RGFClassifier()
kf=StratifiedKFold(n_splits=5)
lg=LGBMClassifier(n_estimators=220, learning_rate=0.05)
cat=CatBoostClassifier()

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
scale=MinMaxScaler()

In [ ]:
clf=VotingClassifier(estimators=[('lg', lg), ('cat', cat),('rf', rf), ('rgf', rgf)], voting='soft', weights=[0.49, 1, 0.54,0.5])

In [ ]:
X.info()

In [ ]:
score=list()
predictions=list()
pipe=make_pipeline(scale, clf)
for Train, Test in kf.split(X,Y):
    
    x_train,x_test,y_train,y_test=X.iloc[Train], X.iloc[Test], Y.iloc[Train], Y.iloc[Test]
    pipe.fit(x_train, y_train)
    sc=pipe.score(x_test,y_test)
    score.append(sc)
    prediction=pipe.predict_proba(df_model)[:,1]
    predictions.append(prediction)
        

In [ ]:
np.mean(score)

0.8097142857142856

# Submission File

In [ ]:
sub=np.mean(predictions, axis=0)

In [ ]:
df_sub=pd.DataFrame(sub)

In [ ]:
df_id=df_test[['Applicant_ID']]
df_sub=pd.concat([df_id,df_sub], axis=1)

In [ ]:
df_sub.head(2)

In [ ]:
from google.colab import files
df_sub.to_csv('./submissionfile.csv', sep = ',', index = False)

In [ ]:
files.download('submissionfile.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>